In [1]:
import numpy as np
import pandas as pd
import mysql.connector

import time

from datetime import datetime, timedelta

In [2]:
def percentage_unique(array):
    return len(set(array))/len(array)


db = mysql.connector.connect(
  host = "hackathon-db.bdc.n360.io",
  user = "events",
  passwd = "Hack@th0n2019"
)

promo_cursor = db.cursor()
promo_cursor.execute('SELECT start_date, end_date, trim_id, model, model_id, payment, rebate, bonus, cashdown FROM hackathon.promotion WHERE trim_id is NOT NULL && start_date is NOT NULL && end_date is NOT NULL')

promo_results = promo_cursor.fetchall()

In [3]:
lead_cursor = db.cursor()
lead_cursor.execute('SELECT lead_id, lead_car_info_id, lead_status, date_created FROM hackathon.lead WHERE date_created is NOT NULL')

lead_results = lead_cursor.fetchall()

In [4]:
lead_car_cursor = db.cursor()
lead_car_cursor.execute('SELECT lead_car_info_id, car_inventory_id FROM hackathon.lead_car_info')

lead_car_results = lead_car_cursor.fetchall()

In [5]:
car_inventory_cursor = db.cursor()
car_inventory_cursor.execute('SELECT inventory_trim_id, car_id FROM hackathon.car_inventory')

car_inventory_results = car_inventory_cursor.fetchall()

In [30]:
set([t for (t, _) in car_inventory_results])

{1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 18

In [7]:
activity_cursor = db.cursor()
activity_cursor.execute('SELECT lead_id, date_done FROM hackathon.activity')

activity_results = activity_cursor.fetchall()

KeyboardInterrupt: 

In [ ]:
promo_results[0][1]

In [ ]:
a = set([b for (_, b) in lead_car_results if b is not None])

In [ ]:
b = [b for (_, b) in car_inventory_results]

In [ ]:
#[promo[2] for promo in promo_results if promo[2] is not None]

### Some statistics:
- Number of clients:  $1\,407\,331$
- Only $36\%$ of ```promo_results``` are retained
- Only $17\%$ unique promotions

In [31]:
dict_trim_id = {_car_id: _trim_id for (_trim_id, _car_id) in car_inventory_results}
dict_car_id  = {_lead_car_info_id: dict_trim_id[car_id] for (_lead_car_info_id, _car_id) in lead_car_results if _car_id is not None}

all_trim_ids  = set([promo[2] for promo in promo_results])
dict_promo    = {trim_id: [promo for promo in promo_results if promo[2] == trim_id] for trim_id in all_trim_ids}

In [12]:
def return_0(array):
    output = []
    
    for elem in array:
        if elem is None:
            output.append(0)
        else:
            output.append(elem)
            
    return output


def transform_outcome(string):
    if 'Sold' in string:
        return 1
    
    return 0


def check_influence(trim_id, date_created):
    keys = ['payment', 'rebate', 'bonus', 'cashdown']
    
    all_zeros = {'exposure': 0,
                 'payment': 0,
                 'rebate': 0,
                 'bonus': 0,
                 'cashdown': 0}
    
    try:
        for promo in list(dict_promo[trim_id].values()):
            start_date = promo[0]
            end_date   = promo[1]
        
            print((start_date, date_created.date()))
        
            if start_date < date_created.date() and date_created.date() < end_date:
                _, _, _, model, model_id, payment, rebate, bonus, cashdown = promo

                quant_characs = [payment, rebate, bonus, cashdown]
                quant_characs = return_0(quant_characs)

                output_dict = {k: q_c for (k, q_c) in list(zip(keys, quant_characs))}
                output_dict['exposure'] = 1

                return output_dict
            
    except Exception as e:
        return all_zeros
    
    return all_zeros


columns = ['outcome',
           'exposure',
           'payment',
           'rebate',
           'bonus',
           'cashdown']

In [14]:
dataframe = pd.DataFrame(columns=columns)
print(columns)

['outcome', 'exposure', 'payment', 'rebate', 'bonus', 'cashdown']


In [32]:
start = time.time()

for i, client in enumerate(lead_results): # every client is unique
    lead_id, lead_car_info_id, lead_status, date_created = client
    
    outcome = transform_outcome(lead_status)
    
    try:
        trim_id  = dict_car_id[lead_car_info_id]
#        trim_id = dict_trim_id[car_id]
        
    except Exception as e:
        continue
    
    print(trim_id)
    _qc = check_influence(trim_id, date_created)
    exposure, payment, rebate, bonus, cashdown = list(_qc.values())
    
    dataframe.loc[i] = [outcome, exposure, payment, rebate, bonus, cashdown]
    

    if i % 10000 == 0:
        
        print('Progress: {}%\t took {} seconds'.format(np.around(i/len(lead_results), 4)*100, np.around(time.time() - start)))
        start = time.time()

266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266
266


KeyboardInterrupt: 